In [15]:
from broverse import Action, Flow, End

In [30]:
from broverse.bedrock import BedrockChat

In [16]:
class Start(Action):
    def __init__(self, message, next_action):
        super().__init__()
        self.message = message
        self.next_action = next_action
        
    def run(self, shared):
        print(self.message)
        shared['action'] = self.next_action
        return shared

In [17]:
from inspect import isfunction, isclass

class Input(Action):
    def run(self, shared):
        user_input = input("You:")
        shared['user_input'] = user_input
        shared['action'] = 'command'
        return shared
    
class Command(Action):
    def command_type(self, user_input:str):
        if user_input.startswith('/clear'):
            return 'clear'
        if user_input.startswith('/exit'):
            return 'end'
        return 'continue'
    
    def run(self, shared):
        user_input = shared.get('user_input', 'no input')
        shared['command'] = self.command_type(user_input)
        action['action'] = 'tool'
        return shared
    
class Tools(Action):
    def __init__(self, tools):
        super().__init__()
        self.tools = self.register_tools(tools)
    
    def register_tools(self, tools):
        self.tools = {}
        for tool in tools:
            name = tool.__name__

    def extract_params(self, shared):
        command = shared['command'].get('command', 'no command')
        user_input = shared.get("user_input", 'no input')
        
        return shared
            
    def run(self, shared):
        shared['user_input'] = ''
        shared['action'] = 'continue'
        return shared

class Tool(Action):
    pass

In [39]:
class Test(Action):
    """abc"""
    pass

def test():
    """def"""
    ...
_test = Test()
_test.__class__, test.__name__

(__main__.Test, 'test')

In [50]:
from inspect import isfunction, isclass

isfunction(test)



True

In [40]:
_test.__doc__, test.__doc__

('abc', 'def')

In [41]:
type(test)

function

In [42]:
type(_test)

__main__.Test

In [47]:
isinstance(_test, Action)

True

In [18]:
input_action = Input()
command_action = Command()
clear_action = Clear()
start_action = Start(message="Start", next_action="user_input")
end_action = End(message="End")

In [19]:
start_action - "user_input" >> input_action
input_action - "command" >> command_action
command_action - "clear" >> clear_action

for action in [command_action, clear_action]:
    action - "continue" >> input_action

command_action - "end" >> end_action

In [20]:
flow = Flow(start_action=start_action)

In [21]:
chart = flow.to_mermaid()
print(chart)

```mermaid
flowchart TD
    Start -->|user_input| Input
    Input -->|command| Command
    Command -->|clear| Clear
    Clear -->|continue| Input
    Command -->|continue| Input
    Command -->|end| End
```


In [27]:
shared={}
flow.run(shared=shared)

Start


You: 
You: 
You: 
You: 
You: 
You: /exit


End


In [28]:
shared

{'action': None, 'user_input': '/exit'}

In [31]:
model = BedrockChat()
system_prompt = "You are a helpful assistant"
messages = [
    model.UserMessage(text="Hi")
]
response = model.run(system_prompt, messages)
response

'How can I assist you today?'

In [ ]:
class CreateCalendar(Action):
    """This agent can create a calendar
    """
    def __init__(self, shared):
        pass
    
    def __call__(self, shared):
        return self.run(shared)
    
    def run(self, shared):
        return shared

What if: 

- router can call agent which has tools

In [226]:
import inspect
import typing
from typing import get_type_hints
import json

class Tools(Action):
    def __init__(self, system_prompt:str, model:BedrockChat, tools:list):
        super().__init__()
        self.system_prompt = system_prompt
        self.model = model
        self.tools = self.register_tools(tools)

    def convert_to_tool(self, func)->dict:
        sig = inspect.signature(func)
        type_hints = get_type_hints(func)

        parameters = {
            "type": "object",
            "properties": {},
            "required": []
        }

        for name, param in sig.parameters.items():
            param_type = type_hints.get(name, str)
            param_schema = {"type": python_type_to_json_type(param_type)}
            parameters["properties"][name] = param_schema
            if param.default is param.empty:
                parameters["required"].append(name)

        tool = {
            "name": func.__name__,
            "description": func.__doc__ or "",
            "parameters": parameters
        }

        return tool


    def python_type_to_json_type(self, py_type):
        origin = typing.get_origin(py_type) or py_type

        if origin in [int]:
            return "integer"
        elif origin in [float]:
            return "number"
        elif origin in [bool]:
            return "boolean"
        elif origin in [str]:
            return "string"
        elif origin in [list, tuple]:
            return "array"
        elif origin in [dict]:
            return "object"
        else:
            return "string"  # fallback
        
    def get_params(self, tool):
        return self.convert_to_tool(tool)
        
    def register_tools(self, tools):
        _tools = {}
        for tool, action in tools:
            tool_definition = self.get_params(tool)
            name = tool_definition.get("name", None)
            _tools[name] = {
                "definition": tool_definition,
                "tool": tool,
                "action": action
            }
        return _tools
    def parse_str_to_json(self, params):
        params = params.split("```json")
        if len(params)>1:
            params = params[1]
        else:
            params = params[0]
        params = params.split("```")[0]
        return json.loads(params)
        
    def run(self, shared):
        user_input = shared.get('user_input', 'no input')
        tool_name = shared.get('tool', 'no tool')
        tool = self.tools.get(tool_name, None)
        definition = tool.get('definition', None)
        action = tool.get('action', None)
        tool = tool.get('tool', None)
        prompt = f"USER_INPUT: \n\n{user_input}\n\n"
        params = self.model.run(system_prompt=self.system_prompt.format(definition=json.dumps(definition)), messages=[self.model.UserMessage(text=prompt)])
        params = self.parse_str_to_json(params)
        params.update({"shared": shared})
        shared['params'] = params
        response = tool(**params)
        shared['tool_res'] = response
        shared['action'] = action
        return shared

def add(a:int, b:int, **kwargs):
    """Add two numbers together"""
    return a+b
    
def clear_user_input(shared:dict, **kwargs):
    """Clear user input"""
    shared['user_input'] = ''
    return shared

In [227]:
tool_prompt = """\
You are a function argument extractor. 
Extract USER_INPUT required to call the function. Return a JSON object that matches the parameters schema exactly. Only include the keys that are required.

Given the following function definition:
DEFINITION
```json
{definition}
```

Respond only with the JSON object.
And the following user input:
""".strip()

In [228]:
# tool_prompt = """\
# You are a function argument extractor. 

# Given the following function definition:
# ```json
# {schemas}
# ```
# """.strip()

tools = Tools(
    system_prompt=tool_prompt,
    model=BedrockChat(),
    tools=[
        (clear_user_input, 'continue'),
        (add, 'continue')
    ]
)

In [229]:
tools.tools

{'clear_user_input': {'definition': {'name': 'clear_user_input',
   'description': 'Clear user input',
   'parameters': {'type': 'object',
    'properties': {'shared': {'type': 'object'}, 'kwargs': {'type': 'string'}},
    'required': ['shared', 'kwargs']}},
  'tool': <function __main__.clear_user_input(shared: dict, **kwargs)>,
  'action': 'continue'},
 'add': {'definition': {'name': 'add',
   'description': 'Add two numbers together',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer'},
     'b': {'type': 'integer'},
     'kwargs': {'type': 'string'}},
    'required': ['a', 'b', 'kwargs']}},
  'tool': <function __main__.add(a: int, b: int, **kwargs)>,
  'action': 'continue'}}

In [230]:
# shared = {
#     "user_input": "What is the result of 1 plus 1?",
#     "tool": "add"
# }

shared = {
    "user_input": "Clear user input for me",
    "tool": "clear_user_input"
}

In [231]:
tools.tools['clear_user_input']

{'definition': {'name': 'clear_user_input',
  'description': 'Clear user input',
  'parameters': {'type': 'object',
   'properties': {'shared': {'type': 'object'}, 'kwargs': {'type': 'string'}},
   'required': ['shared', 'kwargs']}},
 'tool': <function __main__.clear_user_input(shared: dict, **kwargs)>,
 'action': 'continue'}

In [232]:
response = tools.run(shared)

In [233]:
type(response)

dict

In [234]:
print(response['params'])

{'shared': {'user_input': '', 'tool': 'clear_user_input', 'params': {...}, 'tool_res': {...}, 'action': 'continue'}, 'kwargs': 'Clear user input for me'}


In [235]:
type(response['params'])

dict

In [236]:
response['tool_res']

{'user_input': '',
 'tool': 'clear_user_input',
 'params': {'shared': {...}, 'kwargs': 'Clear user input for me'},
 'tool_res': {...},
 'action': 'continue'}

In [237]:
response['user_input']

''